# Data exploration notebook
uitleg hoe tabellen samenhangen:

1. Hoofdconcepten en Entiteiten

- Archer: Bevat gegevens over de boogschutters zoals naam, geslacht, categorie, en vereniging.
- Bow: Informatie over bogen zoals type, handgreep, en status (actief of gearchiveerd).
- Arrowset en Arrow: Bevat gegevens over pijlen en pijlenstellen, inclusief specificaties zoals gewicht en diameter.
- Bowsetup: Gedetailleerde configuratie van een boog, inclusief gerelateerde boog en richtmiddelen (via referenties naar Bow en Sight).

2. Wedstrijden en Schietprestaties

- Match: Bevat gegevens over wedstrijden, inclusief locatie, datum, en specifieke instellingen zoals maximale series en schoten.
- Serie: Een reeks schoten binnen een wedstrijd, met locatie en geschatte afstanden.
- Shot: Gedetailleerde gegevens van individuele schoten, inclusief trefpunt (x, y), waarde, tijd, en gerelateerde hartslagdata. Elke Shot verwijst naar een Serie en kan een specifieke pijl (via Arrow) betreffen.
- Motion: Data over bewegingen tijdens een schot, met een verwijzing naar een Shot.

3. Uitrusting en Configuratie

- Sight: Bevat gegevens over vizieren en hun instellingen.
- SightSetting: Specifieke instellingen voor vizieren, gekoppeld aan een boogsetup en pijlenstel.

4. Rondes en Analyses

- Round en RoundEntry: Gegevens over rondes binnen wedstrijden, met koppelingen naar specifieke matches.
- Filter en FilterSet: Hulpmiddelen voor het filteren en analyseren van data, zoals schoten, wedstrijden of uitrusting.

5. Vragenlijsten en Psychologische Data

- Questionaire en Answer: Voor het bijhouden van antwoorden op vragenlijsten. Elke antwoord is gekoppeld aan een vragenlijst en een vraag.
- Kuiperslist: Mogelijk een specifieke psychologische schaal met ja/nee-vragen (1 of 0).
- Question: Bevat vragen die gebruikt worden in vragenlijsten, met aanvullende metadata zoals kleurinstellingen.

6. Tags en Metadata
- Tag en MatchTag: Tags voor het categoriseren van wedstrijden.
- MatchPhoto: Foto's gekoppeld aan wedstrijden.
- android_metadata: Metadata voor mobiele toepassingen (mogelijk voor internationalisering of versiebeheer).

7. Hartslag en Stressgegevens
- Heartrate: Bevat hartslaggegevens zoals gemiddelde BPM, stressindex en variabiliteit (SDNN, RMSSD, PNN50).


## Relaties Samengevat

- Boogschutter naar Uitrusting: Een boogschutter (Archer) gebruikt een bepaalde boog (Bow), pijlenstel (Arrowset), en boogconfiguratie (Bowsetup).
- Uitrusting naar Wedstrijden: Een boogconfiguratie (Bowsetup) en pijlenstel (Arrowset) worden gebruikt in wedstrijden (Match).
- Wedstrijden naar Data: Een wedstrijd (Match) bevat series (Serie) die uit individuele schoten (Shot) bestaan.
- Data-analyse: Filters (Filter, FilterSet) kunnen worden toegepast op schoten, wedstrijden of andere gegevens.
- Psychologische Data: Psychologische evaluaties worden uitgevoerd via vragenlijsten (Questionaire, Kuiperslist) met antwoorden in Answer.



In [35]:
import sqlite3
import pandas as pd
import os
import pickle
pd.set_option('display.max_columns', None)
database= "./jarno.db"

## stappenplan cleaning en preprocessen

1) tabellen joinen met SQL query
2) opkuisen, decoden, ...
3) save as pickle

## 1. tabellen joinen 

In [ ]:
SELECT 
    -- Match details
    m._id AS match_id,
    m.bowsetup_id,
    m.match_date,
    m.location,
    
    -- Bowsetup details
    bs._id AS bowsetup_id,
    bs.bow_id,
    bs.sight_id,
    
    -- Bow details
    b._id AS bow_id,
    b.name AS bow_name,
    b.type AS bow_type,
    
    -- Sight details
    s._id AS sight_id,
    s.name AS sight_name,
    s.manufacturer AS sight_manufacturer,
    
    -- Tag details
    t._id AS tag_id,
    t.name AS tag_name
    
FROM 
    match AS m
-- Join the 'bowsetup' table
LEFT JOIN bowsetup AS bs ON m.bowsetup_id = bs._id
-- Join the 'bow' table
LEFT JOIN bow AS b ON bs.bow_id = b._id
-- Join the 'sight' table
LEFT JOIN sight AS s ON bs.sight_id = s._id
-- Join the 'matchtag' table
LEFT JOIN matchtag AS mt ON m._id = mt.match_id
-- Join the 'tag' table
LEFT JOIN tag AS t ON mt.tag_id = t._id
;

In [51]:
sql_join_query = """
SELECT 
    -- Match details
    m._id AS match_id,
    m.location,
    m.date,
    m.type,
    m.competition,
    m.face,
    m.distance,
    m.distance_units,
    -- m.maxshotsinserie,
    -- m.maxseries,
    m.bowsetup_id,
    m.arrowset_id,
    m.meteo_weather,
    m.meteo_wind,
    m.notes,
    m.unrecorded_volume,
    m.unrecorded_score,
    -- m._aid
    
    -- Bowsetup details
    bs._id AS bowsetup_id,
    -- bs.*,
    bs.name AS bowsetup_name,
    -- TODO: other fields

    
    -- Bow details
    b._id AS bow_id,
    b.name AS bow_name,
    b.handle,
    b.limbs,
    b.notes AS bow_notes,

    -- Arrowset details
    ars._id AS arrowset_id,
    -- as.*,
    ars.name AS arrowset_name,
    ars.manufacturer,
    ars.type,
    ars.spine,
    ars.shaftlength,
    ars.pointweight,
    ars.diameter,
    ars.notes,

    -- Arrow details
    a._id AS arrow_id,
    -- a.*,
    a.ident as arrow_ident_number,
    a.weight,
    a.bare,
    a.broken,
    
    -- Sight details
    s._id AS sight_id,
    s.name AS sight_name,
    s.notes AS sight_notes,
    
    -- Tag details
    t._id AS tag_id,
    t.tag AS tag
    
FROM 
    match AS m
-- Join the 'bowsetup' table
LEFT JOIN bowsetup AS bs ON m.bowsetup_id = bs._id
-- Join the 'bow' table
LEFT JOIN bow AS b ON bs.bow_id = b._id
-- Join the 'arrowset' table
LEFT JOIN arrowset AS ars ON m.arrowset_id = ars._id
-- Join the 'arrow' table
LEFT JOIN arrow AS a ON ars._id = a.arrowset_id
-- Join the 'sight' table
LEFT JOIN sight AS s ON bs.sight_id = s._id
-- Join the 'matchtag' table
LEFT JOIN matchtag AS mt ON m._id = mt.match_id
-- Join the 'tag' table
LEFT JOIN tag AS t ON mt.tag_id = t._id
;
"""

In [52]:
conn = sqlite3.connect(database)
df_joined = pd.read_sql_query(sql_join_query, conn)
conn.close()

df_joined.head()

,match_id,location,date,type,competition,face,distance,distance_units,bowsetup_id,arrowset_id,meteo_weather,meteo_wind,notes,unrecorded_volume,unrecorded_score,bowsetup_id,bowsetup_name,bow_id,bow_name,handle,limbs,bow_notes,arrowset_id,arrowset_name,manufacturer,type,spine,shaftlength,pointweight,diameter,notes,arrow_id,arrow_ident_number,weight,bare,broken,sight_id,sight_name,sight_notes,tag_id,tag
0,1,kjss outdoor,1649376000,1,0,10,70.0,0,1,1,3.0,2.0,None,None,NaN,1,witte ATF X (outdoor),1,witte ATF-X,wiawis ATF-X,NS-g Wood 70 40,met upgraded alignment blocks,1,x10 sjef,Easton,x10 c3,0.35,30.75,120.0,4.5,None,1,1,None,0,0,1,Shibuya goud,Specs are from Shibuya Ultima RC,NaN,None
1,1,kjss outdoor,1649376000,1,0,10,70.0,0,1,1,3.0,2.0,None,None,NaN,1,witte ATF X (outdoor),1,witte ATF-X,wiawis ATF-X,NS-g Wood 70 40,met upgraded alignment blocks,1,x10 sjef,Easton,x10 c3,0.35,30.75,120.0,4.5,None,2,2,None,0,0,1,Shibuya goud,Specs are from Shibuya Ultima RC,NaN,None
2,1,kjss outdoor,1649376000,1,0,10,70.0,0,1,1,3.0,2.0,None,None,NaN,1,witte ATF X (outdoor),1,witte ATF-X,wiawis ATF-X,NS-g Wood 70 40,met upgraded alignment blocks,1,x10 sjef,Easton,x10 c3,0.35,30.75,120.0,4.5,None,3,3,None,0,0,1,Shibuya goud,Specs are from Shibuya Ultima RC,NaN,None
3,1,kjss outdoor,1649376000,1,0,10,70.0,0,1,1,3.0,2.0,None,None,NaN,1,witte ATF X (outdoor),1,witte ATF-X,wiawis ATF-X,NS-g Wood 70 40,met upgraded alignment blocks,1,x10 sjef,Easton,x10 c3,0.35,30.75,120.0,4.5,None,4,4,None,0,0,1,Shibuya goud,Specs are from Shibuya Ultima RC,NaN,None
4,1,kjss outdoor,1649376000,1,0,10,70.0,0,1,1,3.0,2.0,None,None,NaN,1,witte ATF X (outdoor),1,witte ATF-X,wiawis ATF-X,NS-g Wood 70 40,met upgraded alignment blocks,1,x10 sjef,Easton,x10 c3,0.35,30.75,120.0,4.5,None,5,5,None,0,0,1,Shibuya goud,Specs are from Shibuya Ultima RC,NaN,None


## match table (1850 rows)

In [ ]:
conn = sqlite3.connect(database)
df_match = pd.read_sql_query("SELECT * FROM match", conn)

# andere manier, probleem hier is dat het de kolomnamen niet meeneemt:
""" c = conn.cursor()
c.execute('SELECT * FROM match')
data = c.fetchall()
conn.close()
# Convert the data to a pandas DataFrame
df = pd.DataFrame(data) """

#df_match.head()

df_match['_id'].max()



1850

In [3]:
# drop distance_units, sightadviceaccept_shot_id, time_prep, time_shoot, time_warn, forced_end
df_match = df_match.drop(columns=["distance_units", "sightadviceaccept_shot_id", "time_prep", "time_shoot", "time_warn", "forced_end"])
df_match.head()

,_id,_time_c,_time_u,location,date,type,competition,face,distance,maxshotsinserie,maxseries,bowsetup_id,arrowset_id,meteo_weather,meteo_wind,meteo_winddir,notes,unrecorded_volume,unrecorded_score,archived,_aid
0,1,1649427321,1656668691,kjss outdoor,1649376000,1,0,10,70.0,6,6,1,1,3.0,2.0,NaN,None,None,NaN,0,r1iJvHbkTs7K
1,2,1649487912,1656668681,charleroi,1649462400,1,1,10,70.0,6,6,1,1,1.0,1.0,NaN,None,None,NaN,0,r1iJvHbkTs7K
2,3,1649494467,1656668676,charleroi,1649462400,1,1,10,70.0,6,6,1,1,NaN,NaN,NaN,None,None,NaN,0,r1iJvHbkTs7K
3,4,1649504869,1656668669,charleroi,1649462400,1,1,10,70.0,6,6,1,1,NaN,NaN,NaN,None,None,NaN,0,r1iJvHbkTs7K
4,5,1649510611,1656668662,charleroi,1649462400,1,1,10,70.0,6,6,1,1,NaN,NaN,NaN,None,None,NaN,0,r1iJvHbkTs7K


In [4]:
df_match.isnull().sum()

_id                     0
_time_c                 0
_time_u                 0
location                3
date                    0
type                    0
competition             0
face                    0
distance                0
maxshotsinserie         0
maxseries               0
bowsetup_id             0
arrowset_id             0
meteo_weather         366
meteo_wind           1255
meteo_winddir        1731
notes                 687
unrecorded_volume     645
unrecorded_score     1669
archived                0
_aid                    0
dtype: int64

In [5]:
# _time_c, _time_u, date parsen vanuit unix epoch
df_match['_time_c'] = pd.to_datetime(df_match['_time_c'], unit='s')
df_match['_time_u'] = pd.to_datetime(df_match['_time_u'], unit='s')
df_match['date'] = pd.to_datetime(df_match['date'], unit='s') 

# distance parsen naar int
#df_match['distance'] = df_match['distance'].astype(int) #use floats instead of ints to allow for NaN values

df_match.head()

,_id,_time_c,_time_u,location,date,type,competition,face,distance,maxshotsinserie,maxseries,bowsetup_id,arrowset_id,meteo_weather,meteo_wind,meteo_winddir,notes,unrecorded_volume,unrecorded_score,archived,_aid
0,1,2022-04-08 14:15:21,2022-07-01 09:44:51,kjss outdoor,2022-04-08,1,0,10,70.0,6,6,1,1,3.0,2.0,NaN,None,None,NaN,0,r1iJvHbkTs7K
1,2,2022-04-09 07:05:12,2022-07-01 09:44:41,charleroi,2022-04-09,1,1,10,70.0,6,6,1,1,1.0,1.0,NaN,None,None,NaN,0,r1iJvHbkTs7K
2,3,2022-04-09 08:54:27,2022-07-01 09:44:36,charleroi,2022-04-09,1,1,10,70.0,6,6,1,1,NaN,NaN,NaN,None,None,NaN,0,r1iJvHbkTs7K
3,4,2022-04-09 11:47:49,2022-07-01 09:44:29,charleroi,2022-04-09,1,1,10,70.0,6,6,1,1,NaN,NaN,NaN,None,None,NaN,0,r1iJvHbkTs7K
4,5,2022-04-09 13:23:31,2022-07-01 09:44:22,charleroi,2022-04-09,1,1,10,70.0,6,6,1,1,NaN,NaN,NaN,None,None,NaN,0,r1iJvHbkTs7K


In [6]:
# type, competition, face, meteo_weather, meteo_wind, decoden 
# Mapping dictionaries
type_mapping = {
    1: 'WA Outdoor',
    3: 'WA Indoor',
    0: 'Custom',
    12: 'FITA Outdoor'
}
competition_mapping = {
    0: 'training',
    1: 'competition',
    2: 'tuning'
}
face_mapping = {
    0: 'blank bale',
    10: '122cm',
    2: ''
}
metheo_weather_mapping = {
    0: 'sunny',
    1: 'cloudy with sun',
    2: 'cloudy',
    3: 'light rain',
    4: 'heavy rain',
    5: 'indoor',
}
meteo_wind_mapping = {
    0: '0bft',
    1: '1bft',
    2: '2bft',
    3: '3bft',
    4: '4bft',
    5: '5bft',
    6: '6bft',
    7: '7bft',
    8: '8bft',
    9: '9bft'
}
# Replace numerical values with string labels
df_match['type'] = df_match['type'].replace(type_mapping)
df_match['competition'] = df_match['competition'].replace(competition_mapping)
df_match['face'] = df_match['face'].replace(face_mapping)
df_match['meteo_weather'] = df_match['meteo_weather'].replace(metheo_weather_mapping)
df_match['meteo_wind'] = df_match['meteo_wind'].replace(meteo_wind_mapping)
df_match.head()

,_id,_time_c,_time_u,location,date,type,competition,face,distance,maxshotsinserie,maxseries,bowsetup_id,arrowset_id,meteo_weather,meteo_wind,meteo_winddir,notes,unrecorded_volume,unrecorded_score,archived,_aid
0,1,2022-04-08 14:15:21,2022-07-01 09:44:51,kjss outdoor,2022-04-08,WA Outdoor,training,122cm,70.0,6,6,1,1,light rain,2bft,NaN,None,None,NaN,0,r1iJvHbkTs7K
1,2,2022-04-09 07:05:12,2022-07-01 09:44:41,charleroi,2022-04-09,WA Outdoor,competition,122cm,70.0,6,6,1,1,cloudy with sun,1bft,NaN,None,None,NaN,0,r1iJvHbkTs7K
2,3,2022-04-09 08:54:27,2022-07-01 09:44:36,charleroi,2022-04-09,WA Outdoor,competition,122cm,70.0,6,6,1,1,NaN,NaN,NaN,None,None,NaN,0,r1iJvHbkTs7K
3,4,2022-04-09 11:47:49,2022-07-01 09:44:29,charleroi,2022-04-09,WA Outdoor,competition,122cm,70.0,6,6,1,1,NaN,NaN,NaN,None,None,NaN,0,r1iJvHbkTs7K
4,5,2022-04-09 13:23:31,2022-07-01 09:44:22,charleroi,2022-04-09,WA Outdoor,competition,122cm,70.0,6,6,1,1,NaN,NaN,NaN,None,None,NaN,0,r1iJvHbkTs7K


## round table (67 rows)

In [28]:
conn = sqlite3.connect(database)
df_round = pd.read_sql_query("SELECT * FROM round", conn)


#df_round.head()
df_round['_id'].max()

67

## roundentry table (333 rows)

In [29]:
conn = sqlite3.connect(database)
df_roundentry = pd.read_sql_query("SELECT * FROM roundentry", conn)


#df_roundentry.head()
df_roundentry['_id'].max()

333

In [34]:
df_roundentry.isnull().sum()

_id         0
_time_c     0
_time_u     0
n           0
round_id    0
match_id    0
_aid        0
dtype: int64

## serie table (3135 rows)

In [33]:
conn = sqlite3.connect(database)
df_serie = pd.read_sql_query("SELECT * FROM serie", conn)


df_serie.head()
df_serie['_id'].max()

3135

## shot table (16946 rows)

In [32]:
conn = sqlite3.connect(database)
df_shot = pd.read_sql_query("SELECT * FROM shot", conn)


df_shot.head()
df_shot['_id'].max()

16943

## matchtag table (16 rows)

In [30]:
conn = sqlite3.connect(database)
df_matchtag = pd.read_sql_query("SELECT * FROM matchtag", conn)


#df_matchtag.head()
df_matchtag['_id'].max()

16

## tag table (6 rows)

In [31]:
conn = sqlite3.connect(database)
df_tag = pd.read_sql_query("SELECT * FROM tag", conn)


df_tag.head()
df_tag['_id'].max()

6

## other tables

In [ ]:
#TODO: pipeline van andere tabellen maken
# bowsetup
# bow
# sight

# arrowset
# arrow

# formchange


In [ ]:
#TODO: tabellen joinen en samenvoegen

## save merged dataframe as pickle


In [ ]:
with open ('df_jarno.pkl', 'wb') as file:
    pickle.dump(df_merged_and_cleansed, file)

# open in other files
# with open ('df_jarno.pkl', 'rb') as file:
#     df = pickle.load(file)


## extra (helpers)

In [20]:
# save all database tables to csv files

database = "jarno.db"
conn = sqlite3.connect(database)
cursor = conn.cursor()

# get the list of all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
tables = [table[0] for table in tables]

# save each table to a csv file
for table in tables:
    df = pd.read_sql_query(f"SELECT * FROM {table}", conn)
    df.to_csv(f"csv_tables/{table}.csv", index=False)

conn.close()


